In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import operator

In [19]:
#Exploring some basic elements spanning all players
basic = pd.read_csv('Basic_Stats.csv', encoding='utf8')
playerdict = {}
qualities = ('Name', 'Experience', 'Height (inches)', 'Weight (lbs)', 'Position', 'Current Team')
for i in range(len(basic['Current Status'])):
    if basic['Current Status'][i] == 'Active':
        tor = {}
        for a in qualities:
            tor[a] = basic[a][i]
        playerdict[basic['Player Id'][i]] = tor

## Exploring passing scores (expanded for readability)
![passing.png](passing.png)


In [28]:
#Exploring passing
'''
· TD Pass = 4pts
· Every 25 passing yards = 1pts
· 2pt Passing Conversion = 2pts
· Interceptions Thrown = -2pts
'''
passes = pd.read_csv('Career_Stats_Passing.csv', encoding='utf8')
passdict = {}
qualities = ('Games Played', 'Passes Attempted', 'Completion Percentage', 'Pass Attempts Per Game', 'TD Passes', 'Percentage of TDs per Attempts', 'Passes Longer than 20 Yards', 'Ints', 'Int Rate', 'Passing Yards Per Attempt')
for i in range(len(passes['Team'])):
    if passes['Player Id'][i] in playerdict:
        tor = {}
        for a in qualities:
            tor[a] = float(passes[a][i]) if passes[a][i] != '--' else 0
        if passes['Player Id'][i] in passdict:
            s = passdict[passes['Player Id'][i]]
            d = {}
            for key, val in s.items():
                if key != 'times':
                    passdict[passes['Player Id'][i]][key] = ((s[key]*s['times'])+tor[key])/(s['times']+1)
            passdict[passes['Player Id'][i]]['times'] += 1
        else:
            tor['times'] = 1
            passdict[passes['Player Id'][i]] = tor
            passdict[passes['Player Id'][i]]['times'] = 1
pass_scores = {}
for key, val in passdict.items():
    g, att, p_c, _, TD, p_TD, p_20, i, p_i, py = qualities
    p = passdict[key]
    pass_scores[key] = ((p[att]*p[py]*p[p_c])/(p[g]*100)) + (p[TD]*p[p_TD]*4/(p[g]*100)) + (2*p[p_20]*p[p_c]/(p[g]*100)) - (2*p[i]*p[p_i]/(p[g]*100))
top = sorted(pass_scores.items(), key=operator.itemgetter(1))
top.reverse()
positions = {}
for a in top:
    if playerdict[a[0]]['Position'] not in positions:
        positions[playerdict[a[0]]['Position']] = 1
    else:
        positions[playerdict[a[0]]['Position']] += 1
print(positions)

top_by_position = {}

for key in positions.keys():
    top_by_position[key] = []

for a in top:
    pos = playerdict[a[0]]['Position']
    #Limit it to 12 since there's only 12 people in my league
    if len(top_by_position) < 12:
        top_by_position[pos].append(a)

for key, val in top_by_position.items():
    print('Top', key+'s for passing')
    for a in val[:12]:
        print(playerdict[a[0]]['Current Team'], '-', playerdict[a[0]]['Name'], '\nScore:', a[1])
    print('')

{'QB': 76, 'WR': 35, 'P': 14, 'RB': 11, 'SS': 2, 'NT': 1, 'K': 6, 'CB': 1, 'TE': 1, 'ILB': 1}
Top QBs for passing
New Orleans Saints - Brees, Drew 
Score: 190.9470415973713
Atlanta Falcons - Ryan, Matt 
Score: 176.25137245696396
Washington Redskins - Cousins, Kirk 
Score: 175.09741808695662
Detroit Lions - Stafford, Matthew 
Score: 168.76040173451838
Pittsburgh Steelers - Roethlisberger, Ben 
Score: 167.9310715496562
Indianapolis Colts - Luck, Andrew 
Score: 163.75400502857138
Dallas Cowboys - Prescott, Dak 
Score: 159.10449999999997
New England Patriots - Brady, Tom 
Score: 157.93311046676294
Los Angeles Chargers - Rivers, Philip 
Score: 156.51541969428004
Tampa Bay Buccaneers - Winston, Jameis 
Score: 155.69698125000002
Miami Dolphins - Tannehill, Ryan 
Score: 155.67245200000002
Cincinnati Bengals - Dalton, Andy 
Score: 155.0574910394265

Top WRs for passing
Washington Redskins - Pryor, Terrelle 
Score: 9.949116235294117
Seattle Seahawks - McEvoy, Tanner 
Score: 3.2142857142857144
At

## Exploring Rushing
![rushing.png](rushing.png)

In [27]:
#Exploring rushing
'''
· TD Rush = 6pts
· Every 10 rushing yards = 1pt
· 2pt Rushing Conversion = 2pts
'''
rushes = pd.read_csv('Career_Stats_Rushing.csv', encoding='utf8')
rushdict = {}
qualities = ('Games Played', 'Rushing Yards', 'Rushing TDs')
for i in range(len(rushes['Team'])):
    if rushes['Player Id'][i] in playerdict:
        tor = {}
        for a in qualities:
            tor[a] = float(str(rushes[a][i]).replace(',','')) if rushes[a][i] != '--' else 0
        if rushes['Player Id'][i] in rushdict:
            s = rushdict[rushes['Player Id'][i]]
            d = {}
            for key, val in s.items():
                if key != 'times':
                    rushdict[rushes['Player Id'][i]][key] = ((s[key]*s['times'])+tor[key])/(s['times']+1)
            rushdict[rushes['Player Id'][i]]['times'] += 1
        else:
            tor['times'] = 1
            rushdict[rushes['Player Id'][i]] = tor
            rushdict[rushes['Player Id'][i]]['times'] = 1
rush_scores = {}
for key, val in rushdict.items():
    g, y, TD = qualities
    p = rushdict[key]
    rush_scores[key] = (60*p[TD]+p[y])/(10*p[g])
top = sorted(rush_scores.items(), key=operator.itemgetter(1))
top.reverse()
positions = {}
for a in top:
    if playerdict[a[0]]['Position'] not in positions:
        positions[playerdict[a[0]]['Position']] = 1
    else:
        positions[playerdict[a[0]]['Position']] += 1
print(positions)

top_rush_by_position = {}

for key in positions.keys():
    top_rush_by_position[key] = []

for a in top:
    pos = playerdict[a[0]]['Position']
    #Limit it to 12 since there's only 12 people in my league
    top_rush_by_position[pos].append(a)

for key, val in top_rush_by_position.items():
    print('Top', key+'s for rushing')
    for a in val[:12]:
        print(playerdict[a[0]]['Current Team'], '-', playerdict[a[0]]['Name'], '\nScore:', a[1])
    print('')

{'RB': 126, 'QB': 73, 'WR': 120, 'FB': 15, 'DE': 2, 'NT': 1, 'TE': 26, 'SS': 5, 'P': 24, 'CB': 4, 'K': 2, 'FS': 2, 'DB': 2}
Top RBs for rushing
Dallas Cowboys - Elliott, Ezekiel 
Score: 16.873333333333335
New Orleans Saints - Peterson, Adrian 
Score: 14.282113821138212
Pittsburgh Steelers - Bell, Le'Veon 
Score: 11.925531914893616
Chicago Bears - Howard, Jordan 
Score: 11.153333333333334
Tennessee Titans - Murray, DeMarco 
Score: 10.827380952380953
Buffalo Bills - McCoy, LeSean 
Score: 10.729914529914529
Oakland Raiders - Lynch, Marshawn 
Score: 10.670866141732283
Arizona Cardinals - Johnson, David 
Score: 10.1875
Los Angeles Rams - Gurley, Todd 
Score: 10.175862068965516
Tampa Bay Buccaneers - Martin, Doug 
Score: 9.836842105263159
Indianapolis Colts - Gore, Frank 
Score: 9.725
Cincinnati Bengals - Hill, Jeremy 
Score: 9.568085106382979

Top QBs for rushing
Carolina Panthers - Newton, Cam 
Score: 6.931182795698926
New England Patriots - Brissett, Jacoby 
Score: 4.766666666666667
Buffa

## Exploring Reception

This is basically the same as receiving with some nominal changes

In [26]:
#Exploring reception
'''
· TD Reception = 6pts
· Every 10 receiving yards = 1pt
· 2pt Receiving Conversion = 2pts
'''
receptions = pd.read_csv('Career_Stats_Receiving.csv', encoding='utf8')
recdict = {}
qualities = ('Games Played', 'Receiving Yards', 'Receiving TDs')
for i in range(len(receptions['Team'])):
    if receptions['Player Id'][i] in playerdict:
        tor = {}
        for a in qualities:
            tor[a] = float(str(receptions[a][i]).replace(',','')) if receptions[a][i] != '--' else 0
        if receptions['Player Id'][i] in recdict:
            s = recdict[receptions['Player Id'][i]]
            d = {}
            for key, val in s.items():
                if key != 'times':
                    recdict[receptions['Player Id'][i]][key] = ((s[key]*s['times'])+tor[key])/(s['times']+1)
            recdict[receptions['Player Id'][i]]['times'] += 1
        else:
            tor['times'] = 1
            recdict[receptions['Player Id'][i]] = tor
            recdict[receptions['Player Id'][i]]['times'] = 1
rec_scores = {}
for key, val in recdict.items():
    g, y, TD = qualities
    p = recdict[key]
    rec_scores[key] = (60*p[TD]+p[y])/(10*p[g])
top = sorted(rec_scores.items(), key=operator.itemgetter(1))
top.reverse()
positions = {}
for a in top:
    if playerdict[a[0]]['Position'] not in positions:
        positions[playerdict[a[0]]['Position']] = 1
    else:
        positions[playerdict[a[0]]['Position']] += 1
print(positions)

top_rec_by_position = {}

for key in positions.keys():
    top_rec_by_position[key] = []

for a in top:
    pos = playerdict[a[0]]['Position']
    #Limit it to 12 since there's only 12 people in my league
    top_rec_by_position[pos].append(a)

for key, val in top_rec_by_position.items():
    print('Top', key+'s for receiving')
    for a in val[:12]:
        print(playerdict[a[0]]['Current Team'], '-', playerdict[a[0]]['Name'], '\nScore:', a[1])
    print('')

{'WR': 202, 'TE': 115, 'RB': 125, 'FB': 16, 'QB': 27, 'DE': 4, 'DB': 2, 'CB': 3, 'FS': 3, 'LB': 1}
Top WRs for receiving
New York Giants - Beckham, Odell 
Score: 14.469767441860464
Atlanta Falcons - Jones, Julio 
Score: 12.670886075949367
Cincinnati Bengals - Green, A.J. 
Score: 11.715116279069766
Tampa Bay Buccaneers - Evans, Mike 
Score: 11.299999999999999
Pittsburgh Steelers - Brown, Antonio 
Score: 11.264356435643567
New Orleans Saints - Thomas, Michael 
Score: 11.18
Dallas Cowboys - Bryant, Dez 
Score: 10.970103092783503
Denver Broncos - Thomas, Demaryius 
Score: 10.71683168316832
Pittsburgh Steelers - Bryant, Martavis 
Score: 10.257142857142858
Arizona Cardinals - Fitzgerald, Larry 
Score: 10.212376237623761
New York Giants - Marshall, Brandon 
Score: 10.168263473053893
Jacksonville Jaguars - Robinson, Allen 
Score: 9.883333333333333

Top TEs for receiving
New England Patriots - Gronkowski, Rob 
Score: 11.5625
Seattle Seahawks - Graham, Jimmy 
Score: 9.352380952380953
Los Angeles

## Exploring Kicking
![kicking.png](kicking.png)

In [30]:
#Exploring Kicking
'''
· FG Made (50+ yards) = 5pts
· FG Made (40-49 yards) = 4pts
· FG Made (0-39 yards) = 3pts
· Each PAT Made = 1pt
· FG Missed (any distance) = -1
'''

kicks = pd.read_csv('Career_Stats_Field_Goal_Kickers.csv', encoding='utf8')
kickdict = {}
qualities = ('FGs Attempted', 'FGs Made', 'FGs Made 50+ Yards', 'FGs Made 40-49 Yards', 'Games Played', 'Extra Points Made')
for i in range(len(kicks['Team'])):
    if kicks['Player Id'][i] in playerdict:
        tor = {}
        for a in qualities:
            tor[a] = float(str(kicks[a][i]).replace(',','')) if kicks[a][i] != '--' else 0
        if kicks['Player Id'][i] in kickdict:
            s = kickdict[kicks['Player Id'][i]]
            d = {}
            for key, val in s.items():
                if key != 'times':
                    kickdict[kicks['Player Id'][i]][key] = ((s[key]*s['times'])+tor[key])/(s['times']+1)
            kickdict[kicks['Player Id'][i]]['times'] += 1
        else:
            tor['times'] = 1
            kickdict[kicks['Player Id'][i]] = tor
            kickdict[kicks['Player Id'][i]]['times'] = 1
rec_scores = {}

for key, val in kickdict.items():
    att, f, f50, f40, g, e = qualities
    p = kickdict[key]
    rec_scores[key] = (p[f50]*5 + p[f40]*4 + (p[f] - p[f40] - p[f50])*3 - (p[att] - p[f]) + p[e])/p[g]
top = sorted(rec_scores.items(), key=operator.itemgetter(1))
top.reverse()
positions = {}
for a in top:
    if playerdict[a[0]]['Position'] not in positions:
        positions[playerdict[a[0]]['Position']] = 1
    else:
        positions[playerdict[a[0]]['Position']] += 1
print(positions)

top_kicks_by_position = {}

for key in positions.keys():
    top_kicks_by_position[key] = []

for a in top:
    pos = playerdict[a[0]]['Position']
    #Limit it to 12 since there's only 12 people in my league
    top_kicks_by_position[pos].append(a)

for key, val in top_kicks_by_position.items():
    print('Top', key+'s for Kicking')
    for a in val[:12]:
        print(playerdict[a[0]]['Current Team'], '-', playerdict[a[0]]['Name'], '\nScore:', a[1])
    print('')

{'K': 32, 'P': 1}
Top Ks for Kicking
Baltimore Ravens - Tucker, Justin 
Score: 9.4375
New England Patriots - Gostkowski, Stephen 
Score: 9.125
New Orleans Saints - Lutz, Wil 
Score: 8.8125
Dallas Cowboys - Bailey, Dan 
Score: 8.739583333333334
Washington Redskins - Hopkins, Dustin 
Score: 8.548387096774194
Pittsburgh Steelers - Boswell, Chris 
Score: 8.481481481481481
Seattle Seahawks - Walsh, Blair 
Score: 8.315068493150685
Green Bay Packers - Crosby, Mason 
Score: 8.25625
Kansas City Chiefs - Santos, Cairo 
Score: 8.145833333333334
Philadelphia Eagles - Sturgis, Caleb 
Score: 7.967213114754099
Carolina Panthers - Gano, Graham 
Score: 7.89622641509434
Cleveland Browns - Parkey, Cody 
Score: 7.818181818181818

Top Ps for Kicking
Houston Texans - Lechler, Shane 
Score: 0.02222222222222222

